In [1]:
import os
import json
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [2]:
# establish a connection to the MongoDB database
from pymongo import MongoClient

# connect to your Atlas cluster
client = MongoClient(os.environ["MONGODB_URI"])

In [3]:
# establish a connection to the PostgreSQL database
import psycopg2 as pg

conn = pg.connect(
    dbname=os.environ["POSTGRES_DB"],
    user=os.environ["POSTGRES_USER"],
    password=os.environ["POSTGRES_PASSWORD"],
    host=os.environ["POSTGRES_HOST"]
)
cursor = conn.cursor()

# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers

In [47]:
import openai

openai.api_key = os.environ['OPENAI_APIKEY']
llm_model = "gpt-4o-mini"
# llm_model = "chatgpt-4o-latest"


In [48]:
from langchain.chat_models import ChatOpenAI

In [49]:
# To control the randomness and creativity of the generated
# text by an LLM, use temperature = 0.0
chat = ChatOpenAI(temperature=0.0, model=llm_model, openai_api_key=openai.api_key)


### Clean up class names

In [61]:
template_string_classes = """
You are a highly skilled OWL (Web Ontology Language) ontology engineer. Your task is to assist in the creation, validation, and optimization of OWL ontologies, which are formal representations of knowledge. You have expert knowledge in knowledge engineering, description logic, and semantic web technologies. You also excel at defining classes, properties, and relationships between entities, ensuring logical consistency, and facilitating the sharing of knowledge across different domains.

An OWL ontology is a structured framework used to represent and share knowledge about a particular domain. It consists of classes (concepts), properties (relationships and attributes), and individuals (instances). OWL ontologies allow for the modeling of rich, complex relationships between data in a machine-readable format, enabling advanced reasoning, querying, and inference over that data.
Core OWL Concepts:

    Classes: These represent sets or collections of individuals, typically abstract concepts or types (e.g., "Character," "Weapon," "GameLevel").
    Individuals: Instances of classes (e.g., "Mario" is an individual of the class "Character"; "Sword of Flames" is an individual of the class "Weapon").
    Object Properties: Define relationships between two individuals (e.g., "wieldsWeapon" linking a character to a weapon they use, or "locatedIn" linking a character to a particular game level).
    Datatype Properties: Define relationships between an individual and a data value (e.g., "hasHealthPoints" linking a character to a numeric value representing their health).
    SubClassOf: A relation where one class is a subclass of another, inheriting properties (e.g., "BossCharacter" is a subclass of "Character").
    Equivalence: Used to state that two classes or properties are equivalent (e.g., "MagicWeapon" may be declared equivalent to "SpecialWeapon").
    Disjoint Classes: These are classes that cannot share instances (e.g., "Weapon" and "ConsumableItem" are disjoint classes, meaning an item cannot be both a weapon and a consumable).

OWL Inference and Reasoning:

One of the powerful aspects of OWL is that it allows for reasoning over data. Inference engines can deduce new facts based on the relationships and properties defined in the ontology. For example, if "BossCharacter" is a subclass of "Character" and "Bowser" is an individual of "BossCharacter," it can be inferred that "Bowser" is also an individual of "Character." Additionally, if a property like "wieldsWeapon" is defined, you could infer that "Bowser wields a FireballWeapon" if such an individual and relationship are defined.

Your role is organize and structure a list of classes derived from web data. Classes might be the same but have a slightly different spelling, capitalization, wording, abbreviation or plurality. First, you need to clean up these classes and provide a list of final classes. The final classes should be unique.

Second, you need merge some classes in a single class if that makes ontological sense. For example, GameLevel and Level are the same class and should be merged into a single class. All of the class that have In-Game- , or similar, prefix should drop the prefix and be merged with the class without the prefix.

You are given a list of classes as input. You need to provide a list of tuples as output: ("original class name", "cleaned up class name"). For example, if the input is ["Apple", "apple", "Banana", "Ba-Na-Na", "Fruit"], the output should be [("Apple", "Apple"), ("apple", "Apple"), ("Banana", "Banana"), ("Ba-Na-Na", "Banana"), ("Fruit", "Fruit")].

Original classes are case-insensitive. Cleaned up classes should be capitalized and have no special characters or spaces. Only output tuple if the cleaned up class name is different from the original class name.

Input classes: ```{classes}```

"""

In [62]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string_classes)

In [63]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['classes'], input_types={}, partial_variables={}, template='\nYou are a highly skilled OWL (Web Ontology Language) ontology engineer. Your task is to assist in the creation, validation, and optimization of OWL ontologies, which are formal representations of knowledge. You have expert knowledge in knowledge engineering, description logic, and semantic web technologies. You also excel at defining classes, properties, and relationships between entities, ensuring logical consistency, and facilitating the sharing of knowledge across different domains.\n\nAn OWL ontology is a structured framework used to represent and share knowledge about a particular domain. It consists of classes (concepts), properties (relationships and attributes), and individuals (instances). OWL ontologies allow for the modeling of rich, complex relationships between data in a machine-readable format, enabling advanced reasoning, querying, and inference over that data.\nCore OWL Concept

In [64]:
prompt_template.messages[0].prompt.input_variables

['classes']

In [74]:
cursor.execute("SELECT distinct subclass FROM fandom_entities_clean where subclass not in (select class from fandom_classes) ORDER BY subclass")
classes_list = cursor.fetchall()
classes = '\n'.join([c[0] for c in classes_list])

In [75]:
len(classes_list)

11

In [67]:
fandom_input = prompt_template.format_messages(classes=classes)

In [68]:
# Call the LLM to translate to the style of the customer message
entity_info = chat.invoke(fandom_input)

In [69]:
print(entity_info.content)

Here is the cleaned-up list of classes, represented as tuples of the original class name and the cleaned-up class name:

```python
[
    ("2D Open World Crafting Exploration Farming", "2DOpenWorldCraftingExplorationFarming"),
    ("4X Turn-Based Strategy", "4XTurnBasedStrategy"),
    ("Ability Enhancement", "AbilityEnhancement"),
    ("Ability Item", "AbilityItem"),
    ("Ability Modifier", "AbilityModifier"),
    ("Ability Points", "AbilityPoints"),
    ("Access Card", "AccessCard"),
    ("accessory", "Accessory"),
    ("Achievement Guideline", "AchievementGuideline"),
    ("Action Ability", "ActionAbility"),
    ("Action-adventure", "ActionAdventure"),
    ("Action/Adventure", "ActionAdventure"),
    ("Action-adventure game", "ActionAdventureGame"),
    ("Action-adventure platformer", "ActionAdventurePlatformer"),
    ("Action-adventure Role-playing", "ActionAdventureRolePlaying"),
    ("Action-adventure RPG", "ActionAdventureRPG"),
    ("Action Card", "ActionCard"),
    ("Action Gam

In [70]:
output = [
    ("2D Open World Crafting Exploration Farming", "2DOpenWorldCraftingExplorationFarming"),
    ("4X Turn-Based Strategy", "4XTurnBasedStrategy"),
    ("Ability Enhancement", "AbilityEnhancement"),
    ("Ability Item", "AbilityItem"),
    ("Ability Modifier", "AbilityModifier"),
    ("Ability Points", "AbilityPoints"),
    ("Access Card", "AccessCard"),
    ("accessory", "Accessory"),
    ("Achievement Guideline", "AchievementGuideline"),
    ("Action Ability", "ActionAbility"),
    ("Action-adventure", "ActionAdventure"),
    ("Action/Adventure", "ActionAdventure"),
    ("Action-adventure game", "ActionAdventureGame"),
    ("Action-adventure platformer", "ActionAdventurePlatformer"),
    ("Action-adventure Role-playing", "ActionAdventureRolePlaying"),
    ("Action-adventure RPG", "ActionAdventureRPG"),
    ("Action Card", "ActionCard"),
    ("Action Game", "ActionGame"),
    ("Action-platformer", "ActionPlatformer"),
    ("Action Roguelike", "ActionRoguelike"),
    ("Action role-playing", "ActionRolePlaying"),
    ("Action RPG", "ActionRPG"),
    ("Action-RPG", "ActionRPG"),
    ("Action Shooter", "ActionShooter"),
    ("Action Skill", "ActionSkill"),
    ("activated collectible", "ActivatedCollectible"),
    ("Active Ability", "ActiveAbility"),
    ("Active Item", "ActiveItem"),
    ("Active Skill", "ActiveSkill"),
    ("Active Structure", "ActiveStructure"),
    ("Activist Group", "ActivistGroup"),
    ("additive multiplier", "AdditiveMultiplier"),
    ("adoption event", "AdoptionEvent"),
    ("Advanced Material", "AdvancedMaterial"),
    ("Advanced Quest", "AdvancedQuest"),
    ("Advanced Technology", "AdvancedTechnology"),
    ("Adventure-Action RPG", "AdventureActionRPG"),
    ("Adventure Game", "AdventureGame"),
    ("Adventure Quest", "AdventureQuest"),
    ("AdventureRole-Playing", "AdventureRolePlaying"),
    ("Adventure Romance", "AdventureRomance"),
    ("Adventure RPG", "AdventureRPG"),
    ("Adventure/RPG/Horror Game", "AdventureRPGHorrorGame"),
    ("Agent Namecard", "AgentNamecard"),
    ("Aggressive Enemy", "AggressiveEnemy"),
    ("AI Control Unit", "AIControlUnit"),
    ("AI Entity", "AIEntity"),
    ("Airborne Division", "AirborneDivision"),
    ("aircraft", "Aircraft"),
    ("album", "Album"),
    ("Alien Autopsy", "AlienAutopsy"),
    ("Alien Faction", "AlienFaction"),
    ("Alien Race", "AlienRace"),
    ("Alien Relic", "AlienRelic"),
    ("Alpha-Xenos", "AlphaXenos"),
    ("Altered Item", "AlteredItem"),
    ("Alternate Dimension", "AlternateDimension"),
    ("ammunition", "Ammunition"),
    ("ammunition reserve", "AmmunitionReserve"),
    ("amulet", "Amulet"),
    ("analysis", "Analysis"),
    ("Ancient Technique", "AncientTechnique"),
    ("Animal Companion", "AnimalCompanion"),
    ("Animal Enclosure", "AnimalEnclosure"),
    ("Animal Product", "AnimalProduct"),
    ("Animated Episode", "AnimatedEpisode"),
    ("anniversary", "Anniversary"),
    ("Announcer Pack", "AnnouncerPack"),
    ("anomalous event", "AnomalousEvent"),
    ("Anti-Hero", "AntiHero"),
    ("Any Classes", "AnyClasses"),
    ("Apartment building", "ApartmentBuilding"),
    ("Aquatic Animal", "AquaticAnimal"),
    ("arcade challenge", "ArcadeChallenge"),
    ("Arcane Item", "ArcaneItem"),
    ("Archaeology Event", "ArchaeologyEvent"),
    ("archaeology quest", "ArchaeologyQuest"),
    ("archive", "Archive"),
    ("archive_data", "ArchiveData"),
    ("area", "Area"),
    ("arena", "Arena"),
    ("arm", "Arm"),
    ("arm augmentation", "ArmAugmentation"),
    ("Armed Airship", "ArmedAirship"),
    ("armor", "Armor"),
    ("Armored Division", "ArmoredDivision"),
    ("Armored Transport", "ArmoredTransport"),
    ("Armored Unit", "ArmoredUnit"),
    ("Armored Vehicle", "ArmoredVehicle"),
    ("Armoured Division", "ArmouredDivision"),
    ("Armoured Vehicle", "ArmouredVehicle"),
    ("Army Division", "ArmyDivision"),
    ("Art Asset", "ArtAsset"),
    ("artifact", "Artifact"),
    ("Artificial Intelligence", "ArtificialIntelligence"),
    ("artillery", "Artillery"),
    ("artisan item", "ArtisanItem"),
    ("artisan product", "ArtisanProduct"),
    ("Art Student", "ArtStudent"),
    ("artwork", "Artwork"),
    ("Assault Carrier", "AssaultCarrier"),
    ("Assault Rifle", "AssaultRifle"),
    ("Asset Pack", "AssetPack"),
    ("assignment item", "AssignmentItem"),
    ("assignment items", "AssignmentItems"),
    ("Asterism Pilot", "AsterismPilot"),
    ("Attack Ability", "AttackAbility"),
    ("Attack Bomber", "AttackBomber"),
    ("Attack Card", "AttackCard"),
    ("Attacker Aircraft", "AttackerAircraft"),
    ("attack item", "AttackItem"),
    ("Attack Move", "AttackMove"),
    ("Attack Movement", "AttackMovement"),
    ("Attack Skill", "AttackSkill"),
    ("attack type", "AttackType"),
    ("Attack Upgrade", "AttackUpgrade"),
    ("Attack Vehicle", "AttackVehicle"),
    ("Audio Broadcast", "AudioBroadcast"),
    ("Audio Datapoint", "AudioDatapoint"),
    ("Audio Item", "AudioItem"),
    ("audio show", "AudioShow"),
    ("augmenting item", "AugmentingItem"),
    ("Autonomous Unit", "AutonomousUnit"),
    ("Auto Parts", "AutoParts"),
    ("autopsy", "Autopsy"),
    ("Autopsy Project", "AutopsyProject"),
    ("autopsy report", "AutopsyReport"),
    ("auxiliary", "Auxiliary"),
    ("Back accessory", "BackAccessory"),
    ("backpack", "Backpack"),
    ("Barnyard Animal", "BarnyardAnimal"),
    ("Base Defense", "BaseDefense"),
    ("Base Skill", "BaseSkill"),
    ("Basic Ability", "BasicAbility"),
    ("Basic Lessons", "BasicLessons"),
    ("battery", "Battery"),
    ("battle", "Battle"),
    ("Battle Ability", "BattleAbility"),
    ("Battle Arena Shooter", "BattleArenaShooter"),
    ("Battle Pet", "BattlePet"),
    ("Battle Quest", "BattleQuest"),
    ("Battle Royale", "BattleRoyale"),
    ("Battle Royale Game", "BattleRoyaleGame"),
    ("battleship", "Battleship"),
    ("Battle Skill", "BattleSkill"),
    ("Beginner Mission", "BeginnerMission"),
    ("Behemoth Breakpart", "BehemothBreakpart"),
    ("Bell Bearing", "BellBearing"),
    ("Benedict-Class Sky Battleship", "BenedictClassSkyBattleship"),
    ("Binary Planet", "BinaryPlanet"),
    ("Bio-Android", "BioAndroid"),
    ("biome", "Biome"),
    ("Black Magus", "BlackMagus"),
    ("block", "Block"),
    ("Blood Maiden", "BloodMaiden"),
    ("Blood Youth", "BloodYouth"),
    ("blueprint", "Blueprint"),
    ("Board Game", "BoardGame"),
    ("Boat Ride", "BoatRide"),
    ("Body Augmentation", "BodyAugmentation"),
    ("bond episode", "BondEpisode"),
    ("Bond Mission", "BondMission"),
    ("bonus level", "BonusLevel"),
    ("Bonus Resource", "BonusResource"),
    ("book", "Book"),
    ("Boolean Operator", "BooleanOperator"),
    ("Boost Upgrade", "BoostUpgrade"),
    ("boss", "Boss"),
    ("bossCharacter", "BossCharacter"),
    ("Boss Creature", "BossCreature"),
    ("Boss Defeat", "BossDefeat"),
    ("Boss Encounter", "BossEncounter"),
    ("Boss Enemy", "BossEnemy"),
    ("Boss Event", "BossEvent"),
    ("Boss Level", "BossLevel"),
    ("BOSS LEVEL", "BossLevel"),
    ("boss summon", "BossSummon"),
    ("Breach Bot", "BreachBot"),
    ("Breaching Tool", "BreachingTool"),
    ("Breakable Item", "BreakableItem"),
    ("bridge", "Bridge"),
    ("Bronze Achievement", "BronzeAchievement"),
    ("Bronze Character", "BronzeCharacter"),
    ("Bronze Trophy", "BronzeTrophy"),
    ("Brute Wyvern", "BruteWyvern"),
    ("Buff / Special", "BuffSpecial"),
    ("Buff Spell", "BuffSpell"),
    ("Build & Battle Structure", "BuildAndBattleStructure"),
    ("builder", "Builder"),
    ("building", "Building"),
    ("Building Component", "BuildingComponent"),
    ("Building Improvement", "BuildingImprovement"),
    ("Building Material", "BuildingMaterial"),
    ("Building Service", "BuildingService"),
    ("Building Structure", "BuildingStructure"),
    ("Building Technology", "BuildingTechnology"),
    ("bundle", "Bundle"),
    ("Burrito stand", "BurritoStand"),
    ("Café", "Cafe"),
    ("callsign banner", "CallsignBanner"),
    ("Callsign Patch", "CallsignPatch"),
    ("camp", "Camp"),
    ("Campaign Achievement", "CampaignAchievement"),
    ("Campaign Mission", "CampaignMission"),
    ("campaign token", "CampaignToken"),
    ("Canyon Town", "CanyonTown"),
    ("Capital Ship", "CapitalShip"),
    ("Capsule Machine", "CapsuleMachine"),
    ("car", "Car"),
    ("card", "Card"),
    ("card ability", "CardAbility"),
    ("Card Game", "CardGame"),
    ("cargo", "Cargo"),
    ("cargo airship", "CargoAirship"),
    ("Cargo Enhancer", "CargoEnhancer"),
    ("cargo extender", "CargoExtender"),
    ("Cargo Pod", "CargoPod"),
    ("Cargo Upgrade", "CargoUpgrade"),
    ("Car Manufacturer", "CarManufacturer"),
    ("Car pack", "CarPack"),
    ("case", "Case"),
    ("caster", "Caster"),
    ("catchable creature", "CatchableCreature"),
    ("category", "Category"),
    ("Cave Biome", "CaveBiome"),
    ("challenge", "Challenge"),
    ("Challenge Mission", "ChallengeMission"),
    ("challenge mode", "ChallengeMode"),
    ("Chanter Invocation", "ChanterInvocation"),
    ("chapter", "Chapter"),
    ("Chapter Event", "ChapterEvent"),
    ("character_ability", "CharacterAbility"),
    ("Character Artist", "CharacterArtist"),
    ("Character Enhancement", "CharacterEnhancement"),
    ("Character Skill", "CharacterSkill"),
    ("Chemical Element", "ChemicalElement"),
    ("chest", "Chest"),
    ("Choice-based Adventure", "ChoiceBasedAdventure"),
    ("Chthonic Goddess", "ChthonicGoddess"),
    ("cinematic trailer", "CinematicTrailer"),
    ("city", "City"),
    ("City Builder", "CityBuilder"),
    ("City improvement", "CityImprovement"),
    ("city pack", "CityPack"),
    ("city quest", "CityQuest"),
    ("Civic Doctrine", "CivicDoctrine"),
    ("Civilian car", "CivilianCar"),
    ("client", "Client"),
    ("Close-In Weapon System", "CloseInWeaponSystem"),
    ("clothing", "Clothing"),
    ("Clothing store", "ClothingStore"),
    ("Club Manager", "ClubManager"),
    ("Coalition government", "CoalitionGovernment"),
    ("Code Fragment", "CodeFragment"),
    ("Codex Entry", "CodexEntry"),
    ("Co-founder", "CoFounder"),
    ("coin", "Coin"),
    ("collectible", "Collectible"),
    ("collectible achievement", "CollectibleAchievement"),
    ("collecting quest", "CollectingQuest"),
    ("Colonization Technology", "ColonizationTechnology"),
    ("Colony Simulator", "ColonySimulator"),
    ("Combat Ability", "CombatAbility"),
    ("Combat Achievement", "CombatAchievement"),
    ("Combat Consumable", "CombatConsumable"),
    ("Combat Enhancement", "CombatEnhancement"),
    ("Combat Event", "CombatEvent"),
    ("Combat Mechanic", "CombatMechanic"),
    ("Combat Mod", "CombatMod"),
    ("combat module", "CombatModule"),
    ("Combat Scenario", "CombatScenario"),
    ("Combat Skill", "CombatSkill"),
    ("Combat System", "CombatSystem"),
    ("Combat Technique", "CombatTechnique"),
    ("Combat Technology", "CombatTechnology"),
    ("comic issue", "ComicIssue"),
    ("comic_issue", "ComicIssue"),
    ("Command Ship", "CommandShip"),
    ("Common Area", "CommonArea"),
    ("Common Enemy", "CommonEnemy"),
    ("Common footsoldier", "CommonFootsoldier"),
    ("Community Manager", "CommunityManager"),
    ("company", "Company"),
    ("Company Boss", "CompanyBoss"),
    ("compendium", "Compendium"),
    ("competition", "Competition"),
    ("Competitive Team", "CompetitiveTeam"),
    ("component", "Component"),
    ("Conceptual Puzzle", "ConceptualPuzzle"),
    ("Conflict Request", "ConflictRequest"),
    ("conglomerate", "Conglomerate"),
    ("Console Achievement", "ConsoleAchievement"),
    ("construction material", "ConstructionMaterial"),
    ("Construction platform", "ConstructionPlatform"),
    ("consultant", "Consultant"),
    ("consumable", "Consumable"),
    ("Contact Effect", "ContactEffect"),
    ("container", "Container"),
    ("Content Add-on", "ContentAddon"),
    ("Content Creator", "ContentCreator"),
    ("Content Pack", "ContentPack"),
    ("Content Update", "ContentUpdate"),
    ("Controller Specialist", "ControllerSpecialist"),
    ("Convenience Store", "ConvenienceStore"),
    ("Cooking Challenge", "CookingChallenge"),
    ("Cooking Event", "CookingEvent"),
    ("Cooking Game", "CookingGame"),
    ("Cooldown System", "CooldownSystem"),
    ("Cooperative Play", "CooperativePlay"),
    ("Co-Op Event", "CoOpEvent"),
    ("Co-op Map", "CoopMap"),
    ("Co-op Mission Achievement", "CoopMissionAchievement"),
    ("Core Module", "CoreModule"),
    ("Core Upgrade", "CoreUpgrade"),
    ("cosmetic item", "CosmeticItem"),
    ("Cosmetic Set", "CosmeticSet"),
    ("cosmic event", "CosmicEvent"),
    ("costume", "Costume"),
    ("Council Member", "CouncilMember"),
    ("country", "Country"),
    ("courier", "Courier"),
    ("course", "Course"),
    ("Courting gift", "CourtingGift"),
    ("covenant item", "CovenantItem"),
    ("covert operation", "CovertOperation"),
    ("craftable item", "CraftableItem"),
    ("crafted item", "CraftedItem"),
    ("crafting component", "CraftingComponent"),
    ("Crafting Equipment", "CraftingEquipment"),
    ("Crafting Event", "CraftingEvent"),
    ("Crafting item", "CraftingItem"),
    ("Crafting Machine", "CraftingMachine"),
    ("crafting material", "CraftingMaterial"),
    ("crafting recipe", "CraftingRecipe"),
    ("crafting resource", "CraftingResource"),
    ("crafting station", "CraftingStation"),
    ("Crafting Station Enhancement", "CraftingStationEnhancement"),
    ("Creative Director", "CreativeDirector"),
    ("Creativi-Toy", "CreativiToy"),
    ("Creativi-Toys", "CreativiToys"),
    ("creature", "Creature"),
    ("Creature Catalyst", "CreatureCatalyst"),
    ("Creature Essence", "CreatureEssence"),
    ("Creature Food", "CreatureFood"),
    ("Creature Stable", "CreatureStable"),
    ("Creepy mutant", "CreepyMutant"),
    ("Crew Member", "CrewMember"),
    ("Crew Residence", "CrewResidence"),
    ("crib", "Crib"),
    ("Crime Boss", "CrimeBoss"),
    ("Cryptic Vessel", "CrypticVessel"),
    ("cult", "Cult"),
    ("Cultural Association", "CulturalAssociation"),
    ("cultural_event", "CulturalEvent"),
    ("curio", "Curio"),
    ("currency", "Currency"),
    ("curse", "Curse"),
    ("Customer Review", "CustomerReview"),
    ("Custom Roshan", "CustomRoshan"),
    ("Cut Content", "CutContent"),
    ("cutscene", "Cutscene"),
    ("Cyber rogue shooter", "CyberRogueShooter"),
    ("daily heroic encounter", "DailyHeroicEncounter"),
    ("daily quest", "DailyQuest"),
    ("Damage Dealer", "DamageDealer"),
    ("Damage-inflicting Tool", "DamageInflictingTool"),
    ("damage item", "DamageItem"),
    ("Damage Type", "DamageType"),
    ("Dance Song", "DanceSong"),
    ("Dance Track", "DanceTrack"),
    ("Dark Faction", "DarkFaction"),
    ("data artifact", "DataArtifact"),
    ("Database Server", "DatabaseServer"),
    ("Data Capsule", "DataCapsule"),
    ("Data Drive", "DataDrive"),
    ("Data Hub", "DataHub"),
    ("data manipulation command", "DataManipulationCommand"),
    ("data point", "DataPoint"),
    ("data update", "DataUpdate"),
    ("death_resurrection", "DeathResurrection"),
    ("Death Wish Mission", "DeathWishMission"),
    ("deck", "Deck"),
    ("Décor", "Decor"),
    ("decoration", "Decoration"),
    ("decorative item", "DecorativeItem"),
    ("Defense Item", "DefenseItem"),
    ("Defense Mechanism", "DefenseMechanism"),
    ("Defense Structure", "DefenseStructure"),
    ("Defense Technology", "DefenseTechnology"),
    ("Defensive Ability", "DefensiveAbility"),
    ("defensive building", "DefensiveBuilding"),
    ("Defensive Card", "DefensiveCard"),
    ("defensive item", "DefensiveItem"),
    ("Defensive Structure", "DefensiveStructure"),
    ("delivery event", "DeliveryEvent"),
    ("Delivery Order", "DeliveryOrder"),
    ("delivery quest", "DeliveryQuest"),
    ("Demon Hunter", "DemonHunter"),
    ("Demon Huntress", "DemonHuntress"),
    ("Demon Imp", "DemonImp"),
    ("destructible", "Destructible"),
    ("Diagnosis Machine", "DiagnosisMachine"),
    ("difficulty level", "DifficultyLevel"),
    ("disciplinary action", "DisciplinaryAction"),
    ("discovery", "Discovery"),
    ("dish", "Dish"),
    ("disk", "Disk"),
    ("distraction", "Distraction"),
    ("Divination Card", "DivinationCard"),
    ("DLC Character", "DLCCharacter"),
    ("DLC Pack", "DLCPack"),
    ("DLC Prison", "DLCPrison"),
    ("DLC Quest", "DLCQuest"),
    ("doctrine", "Doctrine"),
    ("document", "Document"),
    ("Dojo Mission", "DojoMission"),
    ("Dome Service", "DomeService"),
    ("Dome Services", "DomeServices"),
    ("Doom Engine Mod", "DoomEngineMod"),
    ("downloadable content", "DownloadableContent"),
    ("draft", "Draft"),
    ("drama_CD", "DramaCD"),
    ("drive", "Drive"),
    ("dungeon", "Dungeon"),
    ("Dungeon Area", "DungeonArea"),
    ("dye", "Dye"),
    ("Ecologist Faction", "EcologistFaction"),
    ("Economic Ability", "EconomicAbility"),
    ("Ecosystem Entity", "EcosystemEntity"),
    ("effect", "Effect"),
    ("Effect item", "EffectItem"),
    ("Effect Modifier", "EffectModifier"),
    ("Elder Demon", "ElderDemon"),
    ("Elder Dragon", "ElderDragon"),
    ("election", "Election"),
    ("Electrical Item", "ElectricalItem"),
    ("Electric Pole", "ElectricPole"),
    ("Electronic Device", "ElectronicDevice"),
    ("electronics", "Electronics"),
    ("Elemental Boss", "ElementalBoss"),
    ("Eliminator Bot", "EliminatorBot"),
    ("Elite Commander", "EliteCommander"),
    ("Elite Enemy", "EliteEnemy"),
    ("Elite Soldier", "EliteSoldier"),
    ("email", "Email"),
    ("Emergency Vehicle", "EmergencyVehicle"),
    ("emoticon", "Emoticon"),
    ("ending", "Ending"),
    ("enemy", "Enemy"),
    ("Enemy Autopsy", "EnemyAutopsy"),
    ("Enemy Creature", "EnemyCreature"),
    ("Energy Cell", "EnergyCell"),
    ("energy core", "EnergyCore"),
    ("Energy Weapon", "EnergyWeapon"),
    ("Entertainment Device", "EntertainmentDevice"),
    ("entertainment item", "EntertainmentItem"),
    ("entity", "Entity"),
    ("Environmental Effect", "EnvironmentalEffect"),
    ("episode", "Episode"),
    ("Equipable Item", "EquipableItem"),
    ("equipment", "Equipment"),
    ("Equipment Upgrade", "EquipmentUpgrade"),
    ("era modifier", "EraModifier"),
    ("Escape Team DLC Item", "EscapeTeamDLCPack"),
    ("Escort Quest", "EscortQuest"),
    ("eSportsTeam", "EsportsTeam"),
    ("Euro-Fighter Coaster", "EuroFighterCoaster"),
    ("event", "Event"),
    ("event item", "EventItem"),
    ("Event Quest", "EventQuest"),
    ("Event Trigger", "EventTrigger"),
    ("Event Venue", "EventVenue"),
    ("evidence", "Evidence"),
    ("Evolved Coromon", "EvolvedCoromon"),
    ("Exiled Dragoon", "ExiledDragoon"),
    ("expansion", "Expansion"),
    ("expansion pack", "ExpansionPack"),
    ("expedition", "Expedition"),
    ("Experimental Animatronic", "ExperimentalAnimatronic"),
    ("exploration quest", "ExplorationQuest"),
    ("explosive", "Explosive"),
    ("exterior", "Exterior"),
    ("extractor", "Extractor"),
    ("facility assault", "FacilityAssault"),
    ("facility modification", "FacilityModification"),
    ("faction", "Faction"),
    ("Faction Quest", "FactionQuest"),
    ("Factory Expansion", "FactoryExpansion"),
    ("family background", "FamilyBackground"),
    ("farm", "Farm"),
    ("Farside Analysis", "FarsideAnalysis"),
    ("Favor Quest", "FavorQuest"),
    ("feat", "Feat"),
    ("Feature Film", "FeatureFilm"),
    ("fencing", "Fencing"),
    ("Fictional Corporation", "FictionalCorporation"),
    ("field tile", "FieldTile"),
    ("Field Tool", "FieldTool"),
    ("Fighter Aircraft", "FighterAircraft"),
    ("Fighting Game", "FightingGame"),
    ("figurine", "Figurine"),
    ("Final Boss", "FinalBoss"),
    ("Fine Chemical", "FineChemical"),
    ("Fire Starting", "FireStarting"),
    ("First Person Shooter", "FirstPersonShooter"),
    ("First-Person Shooter", "FirstPersonShooter"),
    ("fish", "Fish"),
    ("Fishing Line", "FishingLine"),
    ("Fishing Rod", "FishingRod"),
    ("Flooring & Walls", "FlooringAndWalls"),
    ("flora", "Flora"),
    ("Flying Enemy", "FlyingEnemy"),
    ("Focus Skill", "FocusSkill"),
    ("follower", "Follower"),
    ("follower event", "FollowerEvent"),
    ("Follow-up Event", "FollowUpEvent"),
    ("food", "Food"),
    ("Food And Drink", "FoodAndDrink"),
    ("Food Production", "FoodProduction"),
    ("Food Production Building", "FoodProductionBuilding"),
    ("Football Team", "FootballTeam"),
    ("forage request", "ForageRequest"),
    ("Force of Nature", "ForceOfNature"),
    ("Forces of Nature", "ForcesOfNature"),
    ("Foreground Spray", "ForegroundSpray"),
    ("Fragile position", "FragilePosition"),
    ("Freelance Artist", "FreelanceArtist"),
    ("Fruit Bearing Tree", "FruitBearingTree"),
    ("Full-size", "FullSize"),
    ("furniture", "Furniture"),
    ("Furniture Item", "FurnitureItem"),
    ("Furniture / Wall Decoration", "FurnitureWallDecoration"),
    ("Fusion Item", "FusionItem"),
    ("gadget", "Gadget"),
    ("game achievement", "GameAchievement"),
    ("game act", "GameAct"),
    ("game bonus", "GameBonus"),
    ("game chapter", "GameChapter"),
    ("game command", "GameCommand"),
    ("Game Content", "GameContent"),
    ("Game Day", "GameDay"),
    ("game designer", "GameDesigner"),
    ("Game Developer", "GameDeveloper"),
    ("Game Development Studio", "GameDevelopmentStudio"),
    ("Game Director", "GameDirector"),
    ("Game Ending", "GameEnding"),
    ("game episode", "GameEpisode"),
    ("game event", "GameEvent"),
    ("game expansion", "GameExpansion"),
    ("Game Feature", "GameFeature"),
    ("game information", "GameInformation"),
    ("game level", "GameLevel"),
    ("Game Level Access", "GameLevelAccess"),
    ("Game Map", "GameMap"),
    ("game mechanic", "GameMechanic"),
    ("gameMechanic", "GameMechanic"),
    ("Game Mission", "GameMission"),
    ("game mode", "GameMode"),
    ("Gameplay Command", "GameplayCommand"),
    ("Gameplay Content", "GameplayContent"),
    ("Gameplay Event", "GameplayEvent"),
    ("Gameplay Feature", "GameplayFeature"),
    ("Gameplay Gear", "GameplayGear"),
    ("Gameplay Mechanic", "GameplayMechanic"),
    ("Gameplay Mechanics", "GameplayMechanics"),
    ("Gameplay Tutorial", "GameplayTutorial"),
    ("Game Puzzle", "GamePuzzle"),
    ("Gamer Score Achievement", "GamerScoreAchievement"),
    ("Game Scenario", "GameScenario"),
    ("game sequence", "GameSequence"),
    ("Game Studio", "GameStudio"),
    ("Game Theme", "GameTheme"),
    ("Game Tour", "GameTour"),
    ("Game Update", "GameUpdate"),
    ("Gaming Event", "GamingEvent"),
    ("gemstone", "Gemstone"),
    ("General Ability", "GeneralAbility"),
    ("General Achievement", "GeneralAchievement"),
    ("Geographical Area", "GeographicalArea"),
    ("gift", "Gift"),
    ("Glitched Item", "GlitchedItem"),
    ("Glitch Level", "GlitchLevel"),
    ("Gold Trophy", "GoldTrophy"),
    ("governing body", "GoverningBody"),
    ("Greater Demon", "GreaterDemon"),
    ("Guardian Sentry", "GuardianSentry"),
    ("Guardian Spirit", "GuardianSpirit"),
    ("guide", "Guide"),
    ("guild", "Guild"),
    ("Hacking Tool", "HackingTool"),
    ("Hacktivist Group", "HacktivistGroup"),
    ("Handheld Console", "HandheldConsole"),
    ("Handheld Game Console", "HandheldGameConsole"),
    ("hardwired upgrade", "HardwiredUpgrade"),
    ("Harvestable Item", "HarvestableItem"),
    ("Harvest Sprite", "HarvestSprite"),
    ("hat", "Hat"),
    ("Hat Bundle", "HatBundle"),
    ("Haunting Case", "HauntingCase"),
    ("head", "Head"),
    ("headgear", "Headgear"),
    ("Healing Ability", "HealingAbility"),
    ("Healing Item", "HealingItem"),
    ("Heating Technology", "HeatingTechnology"),
    ("Heavy Attack", "HeavyAttack"),
    ("Heavy Bomber", "HeavyBomber"),
    ("Heavy Build", "HeavyBuild"),
    ("Heavy Defense", "HeavyDefense"),
    ("Heavy Weapon", "HeavyWeapon"),
    ("Held Item", "HeldItem"),
    ("hero", "Hero"),
    ("Hero Ability", "HeroAbility"),
    ("Hero Event", "HeroEvent"),
    ("Heroic Ability", "HeroicAbility"),
    ("Hero Skill", "HeroSkill"),
    ("hex", "Hex"),
    ("Hex (Area)", "HexArea"),
    ("Hidden Base", "HiddenBase"),
    ("hidden ending", "HiddenEnding"),
    ("hidden quest", "HiddenQuest"),
    ("Hidden Realm", "HiddenRealm"),
    ("Hidden Trophy", "HiddenTrophy"),
    ("hiding place", "HidingPlace"),
    ("hint", "Hint"),
    ("Hissatsu Move", "HissatsuMove"),
    ("Hissatsu Tactic", "HissatsuTactic"),
    ("Hissatsu Technique", "HissatsuTechnique"),
    ("Holographic Quest", "HolographicQuest"),
    ("Horror Game", "HorrorGame"),
    ("hotfix", "Hotfix"),
    ("Household Item", "HouseholdItem"),
    ("houseware", "Houseware"),
    ("Huge Enemy", "HugeEnemy"),
    ("Human Character", "HumanCharacter"),
    ("Human, Mandragora", "HumanMandragora"),
    ("Human/Mutant", "HumanMutant"),
    ("Human (Player)", "HumanPlayer"),
    ("Hunter Ability", "HunterAbility"),
    ("Immobile Ranged Troop", "ImmobileRangedTroop"),
    ("Immortal Being", "ImmortalBeing"),
    ("Indie action roguelike", "IndieActionRoguelike"),
    ("Indie Adventure", "IndieAdventure"),
    ("Indie AdventurePlatformer", "IndieAdventurePlatformer"),
    ("Indie Game", "IndieGame"),
    ("Indie Platformer", "IndiePlatformer"),
    ("Indie Puzzle Adventure", "IndiePuzzleAdventure"),
    ("Indoor Area", "IndoorArea"),
    ("infantry division", "InfantryDivision"),
    ("Infantry Support Hero", "InfantrySupportHero"),
    ("Infantry Unit", "InfantryUnit"),
    ("Info Item", "InfoItem"),
    ("In-game Achievement", "InGameAchievement"),
    ("in-game action", "InGameAction"),
    ("In-game Ally", "InGameAlly"),
    ("In-game Area", "InGameArea"),
    ("In-game Challenge", "InGameChallenge"),
    ("In-game Computer", "InGameComputer"),
    ("in-game document", "InGameDocument"),
    ("In-game Entity", "InGameEntity"),
    ("in-game event", "InGameEvent"),
    ("In-game Item", "InGameItem"),
    ("In-game Lore", "InGameLore"),
    ("In-game Mechanic", "InGameMechanic"),
    ("In-game organization", "InGameOrganization"),
    ("In-game Puzzle", "InGamePuzzle"),
    ("In-game Quest", "InGameQuest"),
    ("In-game Trophy", "InGameTrophy"),
    ("In-game Vehicle", "InGameVehicle"),
    ("ingredient", "Ingredient"),
    ("Ingredient Provider", "IngredientProvider"),
    ("Interactive Area", "InteractiveArea"),
    ("Interactive building", "InteractiveBuilding"),
    ("Interactive Fiction", "InteractiveFiction"),
    ("Interactive Object", "InteractiveObject"),
    ("Interactive Story", "InteractiveStory"),
    ("Intergovernmental Military Alliance", "IntergovernmentalMilitaryAlliance"),
    ("Interior Piece", "InteriorPiece"),
    ("Intermediate Item", "IntermediateItem"),
    ("Intermission Level", "IntermissionLevel"),
    ("invasion", "Invasion"),
    ("inventory item", "InventoryItem"),
    ("Inverted Coaster", "InvertedCoaster"),
    ("investigation", "Investigation"),
    ("item", "Item"),
    ("Item Fetch Quest", "ItemFetchQuest"),
    ("jewellery", "Jewellery"),
    ("jewelry", "Jewelry"),
    ("journal", "Journal"),
    ("journal entry", "JournalEntry"),
    ("journey", "Journey"),
    ("J-Pop", "JPop"),
    ("J-PopSingle", "JPopSingle"),
    ("Junior Artist", "JuniorArtist"),
    ("Juvenile Creature", "JuvenileCreature"),
    ("key", "Key"),
    ("key item", "KeyItem"),
    ("key quest", "KeyQuest"),
    ("Kick Technique", "KickTechnique"),
    ("Killer Encounter", "KillerEncounter"),
    ("Killing Achievement", "KillingAchievement"),
    ("Knight Faction", "KnightFaction"),
    ("Labour Law", "LabourLaw"),
    ("landmine", "Landmine"),
    ("laptop", "Laptop"),
    ("Large Monster", "LargeMonster"),
    ("Lateral Thinking Puzzle", "LateralThinkingPuzzle"),
    ("leadership", "Leadership"),
    ("leg augmentation", "LegAugmentation"),
    ("legendary challenge", "LegendaryChallenge"),
    ("Legendary Technique", "LegendaryTechnique"),
    ("Legendary Yo-kai", "LegendaryYokai"),
    ("Legend Quest Item", "LegendQuestItem"),
    ("Legion Arm", "LegionArm"),
    ("Lesser Demon", "LesserDemon"),
    ("level", "Level"),
    ("level completion", "LevelCompletion"),
    ("Level editor, 2D Platform game", "LevelEditor2DPlatformGame"),
    ("Level Puzzle", "LevelPuzzle"),
    ("Level Upgrade", "LevelUpgrade"),
    ("Life simulation", "LifeSimulation"),
    ("Life Simulation Game", "LifeSimulationGame"),
    ("Life Simulation Role-playing", "LifeSimulationRolePlaying"),
    ("Light Assault Carrier", "LightAssaultCarrier"),
    ("Light Battlestar", "LightBattlestar"),
    ("Light Source", "LightSource"),
    ("limited item", "LimitedItem"),
    ("limited-time event", "LimitedTimeEvent"),
    ("Living Armor", "LivingArmor"),
    ("Living quarters", "LivingQuarters"),
    ("Loading Screen", "LoadingScreen"),
    ("locomotive", "Locomotive"),
    ("Logic & Processing", "LogicAndProcessing"),
    ("loot", "Loot"),
    ("loot box", "LootBox"),
    ("Loot Chest", "LootChest"),
    ("Loot Item", "LootItem"),
    ("Lord Commander", "LordCommander"),
    ("lore", "Lore"),
    ("Lore Character", "LoreCharacter"),
    ("lore document", "LoreDocument"),
    ("Lore Event", "LoreEvent"),
    ("lore item", "LoreItem"),
    ("magazine", "Magazine"),
    ("Mage Guild", "MageGuild"),
    ("Magic Ability", "MagicAbility"),
    ("Magical Ability", "MagicalAbility"),
    ("Magical Being", "MagicalBeing"),
    ("Magical Creature", "MagicalCreature"),
    ("Magical Realm", "MagicalRealm"),
    ("Magic Attribute", "MagicAttribute"),
    ("magic crystal", "MagicCrystal"),
    ("Magic Item", "MagicItem"),
    ("Magic Spell", "MagicSpell"),
    ("Magic Totem", "MagicTotem"),
    ("main", "Main"),
    ("Main Character", "MainCharacter"),
    ("Main Event", "MainEvent"),
    ("mainframe", "Mainframe"),
    ("mainframe branch", "MainframeBranch"),
    ("Main Hub", "MainHub"),
    ("Main Mission", "MainMission"),
    ("Main Protagonist", "MainProtagonist"),
    ("main quest", "MainQuest"),
    ("Main Scenario", "MainScenario"),
    ("Main Story", "MainStory"),
    ("Major Injury", "MajorInjury"),
    ("Male Octoling", "MaleOctoling"),
    ("Male Partner", "MalePartner"),
    ("Management Sim", "ManagementSim"),
    ("Management/tactical combat", "ManagementTacticalCombat"),
    ("Manufacturing Facility", "ManufacturingFacility"),
    ("map", "Map"),
    ("Marine Animal", "MarineAnimal"),
    ("Martial Art", "MartialArt"),
    ("Marvel Emote", "MarvelEmote"),
    ("mask", "Mask"),
    ("Master Instructor", "MasterInstructor"),
    ("Master Relayer", "MasterRelayer"),
    ("Match Condition", "MatchCondition"),
    ("Match Evaluation", "MatchEvaluation"),
    ("material", "Material"),
    ("materials", "Materials"),
    ("Maze Puzzler", "MazePuzzler"),
    ("Mea Culpa Heart", "MeaCulpaHeart"),
    ("mech", "Mech"),
    ("mecha", "Mecha"),
    ("mechanic", "Mechanic"),
    ("Mechanical Device", "MechanicalDevice"),
    ("Mechanical Enemy", "MechanicalEnemy"),
    ("mechanical item", "MechanicalItem"),
    ("mechanical unit", "MechanicalUnit"),
    ("media", "Media"),
    ("Medical Equipment", "MedicalEquipment"),
    ("Medical Technology", "MedicalTechnology"),
    ("Medium Attack", "MediumAttack"),
    ("Medium Boss", "MediumBoss"),
    ("meeting", "Meeting"),
    ("Mega-spell", "MegaSpell"),
    ("Melee Attack", "MeleeAttack"),
    ("Melee cavalry", "MeleeCavalry"),
    ("Melee Character", "MeleeCharacter"),
    ("Melee infantry", "MeleeInfantry"),
    ("Melee Skill", "MeleeSkill"),
    ("Melee Troop", "MeleeTroop"),
    ("Melee Unit", "MeleeUnit"),
    ("Melee Weapon", "MeleeWeapon"),
    ("memento", "Memento"),
    ("memory", "Memory"),
    ("Memory Sequence", "MemorySequence"),
    ("Mentioned-only location", "MentionedOnlyLocation"),
    ("message", "Message"),
    ("Meta-Human", "MetaHuman"),
    ("Militarist Faction", "MilitaristFaction"),
    ("Military Area", "MilitaryArea"),
    ("Military Building", "MilitaryBuilding"),
    ("Military Division", "MilitaryDivision"),
    ("military fleet", "MilitaryFleet"),
    ("Military Operation", "MilitaryOperation"),
    ("Military Organization", "MilitaryOrganization"),
    ("Military Training Group", "MilitaryTrainingGroup"),
    ("Military Unit", "MilitaryUnit"),
    ("Militia Fleet", "MilitiaFleet"),
    ("Mini-Boss", "MiniBoss"),
    ("Mini-Boss Challenge", "MiniBossChallenge"),
    ("Mini Boss Fight", "MiniBossFight"),
    ("Mining Technology", "MiningTechnology"),
    ("Minor Enemy", "MinorEnemy"),
    ("Minor Faction", "MinorFaction"),
    ("miscellaneous", "Miscellaneous"),
    ("Miscellaneous Event", "MiscellaneousEvent"),
    ("Miscellaneous Item", "MiscellaneousItem"),
    ("Miscellaneous Quest", "MiscellaneousQuest"),
    ("mission", "Mission"),
    ("Mission Area", "MissionArea"),
    ("mission item", "MissionItem"),
    ("Mission Objective", "MissionObjective"),
    ("Mission Opportunity", "MissionOpportunity"),
    ("Mobile Castle", "MobileCastle"),
    ("mobs", "Mobs"),
    ("Mobster Leader", "MobsterLeader"),
    ("modifier", "Modifier"),
    ("module", "Module"),
    ("motorcycle", "Motorcycle"),
    ("move", "Move"),
    ("Movement Ability", "MovementAbility"),
    ("Movement Skill", "MovementSkill"),
    ("movie", "Movie"),
    ("Multiplayer Action Game", "MultiplayerActionGame"),
    ("Multiplayer Map", "MultiplayerMap"),
    ("munition", "Munition"),
    ("Munitions Unit", "MunitionsUnit"),
    ("Muscle Car", "MuscleCar"),
    ("music album", "MusicAlbum"),
    ("musical composition", "MusicalComposition"),
    ("musical item", "MusicalItem"),
    ("musical piece", "MusicalPiece"),
    ("Musical Toy", "MusicalToy"),
    ("musical track", "MusicalTrack"),
    ("music collection", "MusicCollection"),
    ("Music Game", "MusicGame"),
    ("music item", "MusicItem"),
    ("music_performance", "MusicPerformance"),
    ("music track", "MusicTrack"),
    ("Mutant Creature", "MutantCreature"),
    ("Mystery File", "MysteryFile"),
    ("Mythical Being", "MythicalBeing"),
    ("mythical creature", "MythicalCreature"),
    ("Mythological Family", "MythologicalFamily"),
    ("Mythological Group", "MythologicalGroup"),
    ("namecard", "Namecard"),
    ("narrative", "Narrative"),
    ("Narrative Chapter", "NarrativeChapter"),
    ("Natural Fishing Lad", "NaturalFishingLad"),
    ("natural resource", "NaturalResource"),
    ("Nature Club Item", "NatureClubItem"),
    ("Negative Trait", "NegativeTrait"),
    ("Negative Vampire", "NegativeVampire"),
    ("Network Engineer", "NetworkEngineer"),
    ("network node", "NetworkNode"),
    ("Neutral Character", "NeutralCharacter"),
    ("Neutral NPC", "NeutralNPC"),
    ("News Item", "NewsItem"),
    ("Night Springs", "NightSprings"),
    ("Noble House", "NobleHouse"),
    ("Nomadic army", "NomadicArmy"),
    ("Non-interactive Item", "NonInteractiveItem"),
    ("non-lethal explosive", "NonLethalExplosive"),
    ("Non-Playable Character (NPC)", "NonPlayableCharacterNPC"),
    ("Non-Playable Item", "NonPlayableItem"),
    ("Normal Ability", "NormalAbility"),
    ("normal enemy", "NormalEnemy"),
    ("Normal Quest", "NormalQuest"),
    ("Normal Type Character", "NormalTypeCharacter"),
    ("note", "Note"),
    ("Notice Board Quest", "NoticeBoardQuest"),
    ("NPC", "NPC"),
    ("object", "Object"),
    ("objective", "Objective"),
    ("objective-based mission", "ObjectiveBasedMission"),
    ("objectives-based mission", "ObjectivesBasedMission"),
    ("Offensive Ability", "OffensiveAbility"),
    ("Offensive Item", "OffensiveItem"),
    ("Offensive Weapon", "OffensiveWeapon"),
    ("officer", "Officer"),
    ("Official Event", "OfficialEvent"),
    ("Old One", "OldOne"),
    ("Olympian God", "OlympianGod"),
    ("Olympian Goddess", "OlympianGoddess"),
    ("One-handed Weapon", "OneHandedWeapon"),
    ("Ongoing Quest", "OngoingQuest"),
    ("Online Mission", "OnlineMission"),
    ("Online Quest", "OnlineQuest"),
    ("Ooblet Move", "OobletMove"),
    ("optic", "Optic"),
    ("optics", "Optics"),
    ("optional mission", "OptionalMission"),
    ("order", "Order"),
    ("original level", "OriginalLevel"),
    ("Original Soundtrack", "OriginalSoundtrack"),
    ("Original Version", "OriginalVersion"),
    ("Oz Kit", "OzKit"),
    ("Pachislot Game", "PachislotGame"),
    ("pack", "Pack"),
    ("painting", "Painting"),
    ("Party Member", "PartyMember"),
    ("Party Trinket", "PartyTrinket"),
    ("pass", "Pass"),
    ("passive ability", "PassiveAbility"),
    ("Passive Aura", "PassiveAura"),
    ("passive collectible", "PassiveCollectible"),
    ("Passive Item", "PassiveItem"),
    ("Passive Perk", "PassivePerk"),
    ("Passive Skill", "PassiveSkill"),
    ("patch", "Patch"),
    ("patch notes", "PatchNotes"),
    ("pay-per-view event", "PayPerViewEvent"),
    ("Performance Car", "PerformanceCar"),
    ("Pet Rescue Event", "PetRescueEvent"),
    ("photo", "Photo"),
    ("Physical Ability", "PhysicalAbility"),
    ("Picture Book", "PictureBook"),
    ("Pinball Game", "PinballGame"),
    ("Pinball Machine", "PinballMachine"),
    ("Piston Cup Racer", "PistonCupRacer"),
    ("planet", "Planet"),
    ("plant", "Plant"),
    ("Platform Game", "PlatformGame"),
    ("Playable Character", "PlayableCharacter"),
    ("Playable Protagonist", "PlayableProtagonist"),
    ("Player Ability", "PlayerAbility"),
    ("Player Character", "PlayerCharacter"),
    ("plushie", "Plushie"),
    ("pod", "Pod"),
    ("Point and Click Adventure", "PointAndClickAdventure"),
    ("poison", "Poison"),
    ("Pokémon", "Pokemon"),
    ("Pokémon Trainer", "PokemonTrainer"),
    ("Political Coalition", "PoliticalCoalition"),
    ("Political Company", "PoliticalCompany"),
    ("Political Entity", "PoliticalEntity"),
    ("political event", "PoliticalEvent"),
    ("Political Figure", "PoliticalFigure"),
    ("Political Leader", "PoliticalLeader"),
    ("Positive Ability", "PositiveAbility"),
    ("poster", "Poster"),
    ("Posthumous Character", "PosthumousCharacter"),
    ("potion", "Potion"),
    ("Power Card", "PowerCard"),
    ("Power-Generating Building", "PowerGeneratingBuilding"),
    ("power generator", "PowerGenerator"),
    ("Power Skill", "PowerSkill"),
    ("power-up", "PowerUp"),
    ("Power-Up Character", "PowerUpCharacter"),
    ("Power-up Item", "PowerUpItem"),
    ("Precursor Structure", "PrecursorStructure"),
    ("Premium Mod", "PremiumMod"),
    ("Pre-order Quest", "PreOrderQuest"),
    ("Primary Quest", "PrimaryQuest"),
    ("Prison Cell", "PrisonCell"),
    ("private corporation", "PrivateCorporation"),
    ("Private Security", "PrivateSecurity"),
    ("Processing Machine", "ProcessingMachine"),
    ("proclamation", "Proclamation"),
    ("Production Building", "ProductionBuilding"),
    ("Production Facility", "ProductionFacility"),
    ("Production Upgrade", "ProductionUpgrade"),
    ("Professional Player", "ProfessionalPlayer"),
    ("Profile Icon", "ProfileIcon"),
    ("Pro Gamer", "ProGamer"),
    ("progress", "Progress"),
    ("projectile", "Projectile"),
    ("Projectile Modifier", "ProjectileModifier"),
    ("project part", "ProjectPart"),
    ("Pro Player", "ProPlayer"),
    ("Proving Ground Project", "ProvingGroundProject"),
    ("Psychological Horror", "PsychologicalHorror"),
    ("publication", "Publication"),
    ("Puzzle Adventure Platformer", "PuzzleAdventurePlatformer"),
    ("Puzzle Element", "PuzzleElement"),
    ("Puzzle Game", "PuzzleGame"),
    ("Puzzle Hub", "PuzzleHub"),
    ("Puzzle Lesson", "PuzzleLesson"),
    ("Puzzle Level", "PuzzleLevel"),
    ("Puzzle-platform", "PuzzlePlatform"),
    ("Puzzle-platformer", "PuzzlePlatformer"),
    ("Puzzle-Platform Game", "PuzzlePlatformGame"),
    ("Puzzle Room", "PuzzleRoom"),
    ("Puzzle, Strategy", "PuzzleStrategy"),
    ("QA Lead", "QALead"),
    ("quest", "Quest"),
    ("Quest Achievement", "QuestAchievement"),
    ("quest item", "QuestItem"),
    ("quest storyline", "QuestStoryline"),
    ("Quiz Game", "QuizGame"),
    ("race", "Race"),
    ("Race Car", "RaceCar"),
    ("racer", "Racer"),
    ("Race Track", "RaceTrack"),
    ("Racing Car", "RacingCar"),
    ("Racing Event", "RacingEvent"),
    ("Racing Game", "RacingGame"),
    ("Racing Track", "RacingTrack"),
    ("Radio Show", "RadioShow"),
    ("Random Event", "RandomEvent"),
    ("Ranged cavalry", "RangedCavalry"),
    ("Ranged Character", "RangedCharacter"),
    ("Ranged infantry", "RangedInfantry"),
    ("Ranged Troop", "RangedTroop"),
    ("Ranged Weapon", "RangedWeapon"),
    ("Rare Collectible", "RareCollectible"),
    ("Rare Technology", "RareTechnology"),
    ("raw crafting material", "RawCraftingMaterial"),
    ("Raw Material", "RawMaterial"),
    ("reactor", "Reactor"),
    ("reagent", "Reagent"),
    ("Rebel Group", "RebelGroup"),
    ("Recon Vehicle", "ReconVehicle"),
    ("record", "Record"),
    ("recovery request", "RecoveryRequest"),
    ("Recruit Ability", "RecruitAbility"),
    ("refined crafting material", "RefinedCraftingMaterial"),
    ("Region/City", "RegionCity"),
    ("relationship item", "RelationshipItem"),
    ("Relationship quest", "RelationshipQuest"),
    ("Reliable Alchemist", "ReliableAlchemist"),
    ("relic", "Relic"),
    ("Relic Crafting Material", "RelicCraftingMaterial"),
    ("relic piece", "RelicPiece"),
    ("Religious organization", "ReligiousOrganization"),
    ("Repair Bay", "RepairBay"),
    ("request", "Request"),
    ("requests", "Requests"),
    ("Research Centre", "ResearchCentre"),
    ("Research Facility", "ResearchFacility"),
    ("Research Item", "ResearchItem"),
    ("Research Project", "ResearchProject"),
    ("Research & Records / Reports", "ResearchAndRecordsReports"),
    ("research technology", "ResearchTechnology"),
    ("Residential home", "ResidentialHome"),
    ("Residential Structure", "ResidentialStructure"),
    ("Resident Quest", "ResidentQuest"),
    ("resource", "Resource"),
    ("Resource Collector", "ResourceCollector"),
    ("Resource Gathering Building", "ResourceGatheringBuilding"),
    ("Resource Generator", "ResourceGenerator"),
    ("Resource Management", "ResourceManagement"),
    ("Resource Producer", "ResourceProducer"),
    ("Resource Production", "ResourceProduction"),
    ("Resource Production Building", "ResourceProductionBuilding"),
    ("restaurant", "Restaurant"),
    ("Retail Shop", "RetailShop"),
    ("Retro-FPS", "RetroFPS"),
    ("Rhythm Game", "RhythmGame"),
    ("ring", "Ring"),
    ("riot", "Riot"),
    ("robot", "Robot"),
    ("Rogue-like", "RogueLike"),
    ("Rogue-like CCG", "RogueLikeCCG"),
    ("Roguelike deck-builder", "RoguelikeDeckBuilder"),
    ("Roguelite Farming Simulator", "RogueliteFarmingSimulator"),
    ("Rogue-liteStrategyTacticalRPG", "RogueLiteStrategyTacticalRPG"),
    ("Role-Playing Game", "RolePlayingGame"),
    ("Roller Coaster", "RollerCoaster"),
    ("ronin mission", "RoninMission"),
    ("Rosary Bead", "RosaryBead"),
    ("Royal Guard", "RoyalGuard"),
    ("RPG Strategy", "RPGStrategy"),
    ("rundown", "Rundown"),
    ("Safari Animal", "SafariAnimal"),
    ("Saga Quest", "SagaQuest"),
    ("Sale Box", "SaleBox"),
    ("Saving Mechanism", "SavingMechanism"),
    ("scale model", "ScaleModel"),
    ("scenario", "Scenario"),
    ("scenario alteration", "ScenarioAlteration"),
    ("Science Fiction", "ScienceFiction"),
    ("Scout Transport", "ScoutTransport"),
    ("Sea Creature", "SeaCreature"),
    ("seasonal_event", "SeasonalEvent"),
    ("Season Episode", "SeasonEpisode"),
    ("secondary", "Secondary"),
    ("Secondary Character", "SecondaryCharacter"),
    ("secondary mission", "SecondaryMission"),
    ("Secondary Objective", "SecondaryObjective"),
    ("Secondary Quest", "SecondaryQuest"),
    ("Secondary Weapon", "SecondaryWeapon"),
    ("Secret Passive", "SecretPassive"),
    ("Secret Room", "SecretRoom"),
    ("Security Feature", "SecurityFeature"),
    ("Security force", "SecurityForce"),
    ("Security Officer", "SecurityOfficer"),
    ("seed", "Seed"),
    ("Server Command", "ServerCommand"),
    ("Settlement Upgrade", "SettlementUpgrade"),
    ("ship", "Ship"),
    ("Ship Equipment", "ShipEquipment"),
    ("Ship Upgrade", "ShipUpgrade"),
    ("Shoot 'em up", "ShootEmUp"),
    ("Shooting Game", "ShootingGame"),
    ("shop", "Shop"),
    ("Shop Quest", "ShopQuest"),
    ("Shops and Facilities", "ShopsAndFacilities"),
    ("short film", "ShortFilm"),
    ("Short range weapon", "ShortRangeWeapon"),
    ("Short Task", "ShortTask"),
    ("Sibling Relationship", "SiblingRelationship"),
    ("side mission", "SideMission"),
    ("Side Pursuit", "SidePursuit"),
    ("side quest", "SideQuest"),
    ("Silver Trophy", "SilverTrophy"),
    ("Sim Trait", "SimTrait"),
    ("Simulation Game", "SimulationGame"),
    ("single", "Single"),
    ("Single Rail Coaster", "SingleRailCoaster"),
    ("skill", "Skill"),
    ("Skill Node", "SkillNode"),
    ("Skill Tree", "SkillTree"),
    ("skin", "Skin"),
    ("Skin Introduction", "SkinIntroduction"),
    ("SKULL", "Skull"),
    ("Small Enemy", "SmallEnemy"),
    ("small object", "SmallObject"),
    ("Smithy Material", "SmithyMaterial"),
    ("Sniper Unit", "SniperUnit"),
    ("Social Media Company", "SocialMediaCompany"),
    ("song", "Song"),
    ("soundtrack", "Soundtrack"),
    ("souvenir", "Souvenir"),
    ("Space Agency", "SpaceAgency"),
    ("Space flight simulation", "SpaceFlightSimulation"),
    ("Space Navigation System", "SpaceNavigationSystem"),
    ("Space Ship", "SpaceShip"),
    ("Space Station", "SpaceStation"),
    ("Space Vessel", "SpaceVessel"),
    ("special", "Special"),
    ("Special Ability", "SpecialAbility"),
    ("special area", "SpecialArea"),
    ("Special Campaign", "SpecialCampaign"),
    ("Special Effect Item", "SpecialEffectItem"),
    ("Special Enemy", "SpecialEnemy"),
    ("special event", "SpecialEvent"),
    ("Special Forces Soldier", "SpecialForcesSoldier"),
    ("Special Gift", "SpecialGift"),
    ("Special Infected", "SpecialInfected"),
    ("special item", "SpecialItem"),
    ("Specialized Unit", "SpecializedUnit"),
    ("Special landmark", "SpecialLandmark"),
    ("Special Landmark Building", "SpecialLandmarkBuilding"),
    ("Special Machine", "SpecialMachine"),
    ("Special Module", "SpecialModule"),
    ("Special Move", "SpecialMove"),
    ("Special Police Officer", "SpecialPoliceOfficer"),
    ("special technique", "SpecialTechnique"),
    ("species", "Species"),
    ("spell", "Spell"),
    ("Spell Behavior", "SpellBehavior"),
    ("Spell Enhancement", "SpellEnhancement"),
    ("spell modifier", "SpellModifier"),
    ("Sport Equipment", "SportEquipment"),
    ("Sports Car", "SportsCar"),
    ("Sports Game", "SportsGame"),
    ("Sports Sim", "SportsSim"),
    ("staff", "Staff"),
    ("Stage of Grief", "StageOfGrief"),
    ("Star Destroyer", "StarDestroyer"),
    ("Star System", "StarSystem"),
    ("Starting Area", "StartingArea"),
    ("Stat Boost", "StatBoost"),
    ("Stationary Device", "StationaryDevice"),
    ("Steel Coaster", "SteelCoaster"),
    ("sticker", "Sticker"),
    ("Storage Component", "StorageComponent"),
    ("Storage Container", "StorageContainer"),
    ("storage item", "StorageItem"),
    ("store", "Store"),
    ("Story Achievement", "StoryAchievement"),
    ("Story Arc", "StoryArc"),
    ("story chapter", "StoryChapter"),
    ("story element", "StoryElement"),
    ("story event", "StoryEvent"),
    ("story item", "StoryItem"),
    ("storyline element", "StorylineElement"),
    ("storyline quest", "StorylineQuest"),
    ("story mission", "StoryMission"),
    ("story quality", "StoryQuality"),
    ("story quest", "StoryQuest"),
    ("Story Sequence", "StorySequence"),
    ("Strategic Cruiser", "StrategicCruiser"),
    ("Strategy Game", "StrategyGame"),
    ("Strategy RPG", "StrategyRPG"),
    ("Strategy Simulation", "StrategySimulation"),
    ("Street gang", "StreetGang"),
    ("Strike Craft", "StrikeCraft"),
    ("structure", "Structure"),
    ("Sub Battlefield", "SubBattlefield"),
    ("Sub-boss", "SubBoss"),
    ("Sub-clone", "SubClone"),
    ("sub-event", "SubEvent"),
    ("sub mission", "SubMission"),
    ("Subordinate Function", "SubordinateFunction"),
    ("Sub Quest", "SubQuest"),
    ("Suburban Town", "SuburbanTown"),
    ("Summon Unit", "SummonUnit"),
    ("Super Carrier", "SuperCarrier"),
    ("Supernatural Event", "SupernaturalEvent"),
    ("Supply Cache", "SupplyCache"),
    ("Supply Quest", "SupplyQuest"),
    ("Support Ability", "SupportAbility"),
    ("support card", "SupportCard"),
    ("Support Item", "SupportItem"),
    ("Support Skill Gem", "SupportSkillGem"),
    ("Surreal Area", "SurrealArea"),
    ("Survival Battle Royale", "SurvivalBattleRoyale"),
    ("Survival Horror", "SurvivalHorror"),
    ("symbol", "Symbol"),
    ("Syndicate building", "SyndicateBuilding"),
    ("system", "System"),
    ("System Access Device", "SystemAccessDevice"),
    ("system component", "SystemComponent"),
    ("Tactical RPG", "TacticalRPG"),
    ("tactical shield", "TacticalShield"),
    ("Tactical Strategy Game", "TacticalStrategyGame"),
    ("Tactical Team Leader", "TacticalTeamLeader"),
    ("Tao Alchemist", "TaoAlchemist"),
    ("task", "Task"),
    ("Task Force", "TaskForce"),
    ("team", "Team"),
    ("technological item", "TechnologicalItem"),
    ("technology", "Technology"),
    ("Technology Faction", "TechnologyFaction"),
    ("technology item", "TechnologyItem"),
    ("Tech Research", "TechResearch"),
    ("Tech Upgrade", "TechUpgrade"),
    ("Teleportation Device", "TeleportationDevice"),
    ("Teleportation Skill", "TeleportationSkill"),
    ("Television Episode", "TelevisionEpisode"),
    ("Temple Ritual", "TempleRitual"),
    ("temtem", "Temtem"),
    ("terrain", "Terrain"),
    ("Territorial Faction", "TerritorialFaction"),
    ("Terrorist Religious Organization", "TerroristReligiousOrganization"),
    ("test version", "TestVersion"),
    ("text datapoint", "TextDatapoint"),
    ("text_datapoint", "TextDatapoint"),
    ("text datapoints", "TextDatapoints"),
    ("text item", "TextItem"),
    ("text mail", "TextMail"),
    ("theatrical adaptation", "TheatricalAdaptation"),
    ("theological writings", "TheologicalWritings"),
    ("Thrill Ride", "ThrillRide"),
    ("Throwing Object", "ThrowingObject"),
    ("Tier 1 unit", "Tier1Unit"),
    ("Tier 2 unit", "Tier2Unit"),
    ("Tier Item", "TierItem"),
    ("time-limited event", "TimeLimitedEvent"),
    ("timeline", "Timeline"),
    ("Timeline Event", "TimelineEvent"),
    ("Titan Ability", "TitanAbility"),
    ("Titan Primary Weapon", "TitanPrimaryWeapon"),
    ("tool", "Tool"),
    ("Total Conversion", "TotalConversion"),
    ("tour", "Tour"),
    ("Touring Car", "TouringCar"),
    ("tournament", "Tournament"),
    ("tower", "Tower"),
    ("Town Service", "TownService"),
    ("Town Services", "TownServices"),
    ("Townsfolk", "Townsfolk"),
    ("Townspeople", "Townspeople"),
    ("Toxic Ability", "ToxicAbility"),
    ("TrackInformation", "TrackInformation"),
    ("Track Ride", "TrackRide"),
    ("Trade Building", "TradeBuilding"),
    ("Trader", "Trader"),
    ("Trading Company", "TradingCompany"),
    ("training", "Training"),
    ("Training Dummy", "TrainingDummy"),
    ("Training Facility", "TrainingFacility"),
    ("Train Station", "TrainStation"),
    ("transcript", "Transcript"),
    ("Transformation Block", "TransformationBlock"),
    ("TransportAircraft", "TransportAircraft"),
    ("Transport Ship", "TransportShip"),
    ("Transportation", "Transportation"),
    ("Transport Belt", "TransportBelt"),
    ("TransportFacility", "TransportFacility"),
    ("Transport item", "TransportItem"),
    ("trap", "Trap"),
    ("Trapper", "Trapper"),
    ("Traveler", "Traveler"),
    ("treasure", "Treasure"),
    ("treasure card", "TreasureCard"),
    ("treasure hunt", "TreasureHunt"),
    ("Tree Fruit", "TreeFruit"),
    ("Triad", "Triad"),
    ("Trick", "Trick"),
    ("Trickery Card", "TrickeryCard"),
    ("Trickster", "Trickster"),
    ("trigger_event", "TriggerEvent"),
    ("triggering event", "TriggeringEvent"),
    ("TriggerSpell", "TriggerSpell"),
    ("trinket", "Trinket"),
    ("trophy", "Trophy"),
    ("trophy challenge", "TrophyChallenge"),
    ("trophy event", "TrophyEvent"),
    ("Truck", "Truck"),
    ("Turf Leader", "TurfLeader"),
    ("Turn-based Strategy", "TurnBasedStrategy"),
    ("Turn-based tactics", "TurnBasedTactics"),
    ("TutorialAchievement", "TutorialAchievement"),
    ("Tutorial Quest", "TutorialQuest"),
    ("UFO", "UFO"),
    ("Ultimate", "Ultimate"),
    ("UndeadEnemy", "UndeadEnemy"),
    ("Under the Sea Land", "UnderTheSeaLand"),
    ("Underworld Army", "UnderworldArmy"),
    ("Unique Ability", "UniqueAbility"),
    ("unique plant", "UniquePlant"),
    ("Unique Trait", "UniqueTrait"),
    ("unit", "Unit"),
    ("Unit Card", "UnitCard"),
    ("Unit Essence", "UnitEssence"),
    ("Unit Training Facility", "UnitTrainingFacility"),
    ("University Organization", "UniversityOrganization"),
    ("Unknown", "Unknown"),
    ("UnknownCharacter", "UnknownCharacter"),
    ("Unknown Entity", "UnknownEntity"),
    ("unknown event", "UnknownEvent"),
    ("Unknown Item", "UnknownItem"),
    ("UnlockableCharacter", "UnlockableCharacter"),
    ("Unlockable Character", "UnlockableCharacter"),
    ("unlockable event", "UnlockableEvent"),
    ("Unlockable Item", "UnlockableItem"),
    ("unlockable structure", "UnlockableStructure"),
    ("Unlock Condition", "UnlockCondition"),
    ("unlock event", "UnlockEvent"),
    ("UnlockingChallenge", "UnlockingChallenge"),
    ("Unlocking Mechanism", "UnlockingMechanism"),
    ("unlock objective", "UnlockObjective"),
    ("UnofficialEvent", "UnofficialEvent"),
    ("Unseen Antagonist", "UnseenAntagonist"),
    ("update", "Update"),
    ("upgrade", "Upgrade"),
    ("Upgrade Component", "UpgradeComponent"),
    ("Upgrade Item", "UpgradeItem"),
    ("Usable Item", "UsableItem"),
    ("utility", "Utility"),
    ("utility item", "UtilityItem"),
    ("Utility Perk", "UtilityPerk"),
    ("Utility Tool", "UtilityTool"),
    ("valuable", "Valuable"),
    ("valuable item", "ValuableItem"),
    ("valuables", "Valuables"),
    ("Vampire", "Vampire"),
    ("Van", "Van"),
    ("vanity item", "VanityItem"),
    ("Vanity Set", "VanitySet"),
    ("Vault", "Vault"),
    ("Vegetation", "Vegetation"),
    ("vehicle", "Vehicle"),
    ("Vehicular Combat", "VehicularCombat"),
    ("Vending Machine", "VendingMachine"),
    ("VenomousCreature", "VenomousCreature"),
    ("version", "Version"),
    ("Versus Mode", "VersusMode"),
    ("VFX Artist", "VFXArtist"),
    ("Video Game", "VideoGame"),
    ("VideoGameDesigner", "VideoGameDesigner"),
    ("Video Game Developer", "VideoGameDeveloper"),
    ("VideoGameMusic", "VideoGameMusic"),
    ("Video Game Producer", "VideoGameProducer"),
    ("Video Game Publisher", "VideoGamePublisher"),
    ("Video Game Soundtrack", "VideoGameSoundtrack"),
    ("video short", "VideoShort"),
    ("Vigilante", "Vigilante"),
    ("village", "Village"),
    ("Village Development Quest", "VillageDevelopmentQuest"),
    ("Villager", "Villager"),
    ("Villain", "Villain"),
    ("Villainous Faction", "VillainousFaction"),
    ("VIP Visitor", "VIPVisitor"),
    ("Virtual Server", "VirtualServer"),
    ("Visionary", "Visionary"),
    ("visor cosmetic", "VisorCosmetic"),
    ("Visual", "Visual"),
    ("Visual Novel", "VisualNovel"),
    ("Voice Actor", "VoiceActor"),
    ("volume", "Volume"),
    ("Wall", "Wall"),
    ("Wall Item", "WallItem"),
    ("Wallpaper", "Wallpaper"),
    ("Wandering NPC", "WanderingNPC"),
    ("war", "War"),
    ("Warlock Card", "WarlockCard"),
    ("Warlord", "Warlord"),
    ("War Machine", "WarMachine"),
    ("Warrior", "Warrior"),
    ("Warrior / Tank", "WarriorTank"),
    ("Watercraft", "Watercraft"),
    ("Waystation", "Waystation"),
    ("weapon", "Weapon"),
    ("Weapon Attachment", "WeaponAttachment"),
    ("WeaponEnhancement", "WeaponEnhancement"),
    ("Weapon Mod", "WeaponMod"),
    ("Weapon Modifier", "WeaponModifier"),
    ("Weapon Platform", "WeaponPlatform"),
    ("Weapon Skin", "WeaponSkin"),
    ("WeaponSkinBundle", "WeaponSkinBundle"),
    ("Weapon System", "WeaponSystem"),
    ("Weapon Technology", "WeaponTechnology"),
    ("wearable", "Wearable"),
    ("Wearable Item", "WearableItem"),
    ("Weekly Schedule", "WeeklySchedule"),
    ("Werewolf", "Werewolf"),
    ("Wheels", "Wheels"),
    ("Widget", "Widget"),
    ("Wielder", "Wielder"),
    ("Wilderness", "Wilderness"),
    ("Wishstone", "Wishstone"),
    ("Witch", "Witch"),
    ("wonders of the world", "WondersOfTheWorld"),
    ("Work Producing Machine", "WorkProducingMachine"),
    ("workstation", "Workstation"),
    ("world", "World"),
    ("World Area", "WorldArea"),
    ("world quest", "WorldQuest"),
    ("Wraith", "Wraith"),
    ("Wraith Lair", "WraithLair"),
    ("Wrestler", "Wrestler"),
    ("Xenos", "Xenos"),
    ("Xeno-tech Artifact", "XenoTechArtifact"),
    ("YearInGame", "YearInGame"),
    ("Yo-kai", "Yokai"),
    ("Yōkai", "Yokai"),
    ("Zee Event", "ZeeEvent"),
    ("ZooAnimal", "ZooAnimal"),
    ("Zookeeper Animal", "ZookeeperAnimal")
]

In [71]:
# Create classes

for o in output:
    cursor.execute("SELECT id FROM fandom_classes WHERE class = %s", (o[1],))
    class_id = cursor.fetchone()
    if class_id is None:
        cursor.execute("INSERT INTO fandom_classes (class) VALUES (%s)", (o[1],))
        conn.commit()
    if o[0] != o[1]:
        cursor.execute("UPDATE fandom_entities_clean SET subclass = %s WHERE subclass = %s", (o[1], o[0]))
        # if cursor.rowcount > 0:
        #     print(f"NUmber of rows affected: {cursor.rowcount}")
        conn.commit()



In [55]:
# Update subclass of:

for o in output:
    if o[2] is not None:
        cursor.execute("SELECT id FROM fandom_classes WHERE class = %s", (o[2],))
        class_id = cursor.fetchone()
        if class_id is not None:
            cursor.execute("UPDATE fandom_classes SET subclassof = %s WHERE class = %s", (class_id[0], o[1]))
            conn.commit()


In [39]:
# conn.rollback()